# Projeto 01: Um Bilhão de Linhas: Desafio de Processamento de Dados com Python

![imagem_01](./img/bootcamp.jpg)

## Introdução

O objetivo deste projeto é demonstrar como processar eficientemente um arquivo de dados massivo contendo 1 bilhão de linhas (~14GB), especificamente para calcular estatísticas (Incluindo agregação e ordenação que são operações pesadas) utilizando Python. 

Este desafio foi inspirado no [The One Billion Row Challenge](https://github.com/gunnarmorling/1brc), originalmente proposto para Java, foi adaptado para o [bootcamp da jornada de dados 2024](https://www.jornadadedados2024.com.br/workshops)

O arquivo de dados consiste em medições de temperatura de várias cidades(dados ficticios). Cada registro segue o formato `<string: nome da estação>;<double: medição>`, com a temperatura sendo apresentada com precisão de uma casa decimal.

Aqui estão dez linhas de exemplo do arquivo:

```
Hamburg;12.0
Bulawayo;8.9
Palembang;38.8
St. Johns;15.2
Cracow;12.6
Bridgetown;26.9
Istanbul;6.2
Roseau;34.4
Conakry;31.2
Istanbul;23.0
```

O desafio é desenvolver um programa Python capaz de ler esse arquivo e calcular a temperatura mínima, média (arredondada para uma casa decimal) e máxima para cada cidade, exibindo os resultados em uma tabela ordenada por nome da estação.

| city      | min_temperature | mean_temperature | max_temperature |
|--------------|-----------------|------------------|-----------------|
| Abha         | -31.1           | 18.0             | 66.5            |
| Abidjan      | -25.9           | 26.0             | 74.6            |
| Abéché       | -19.8           | 29.4             | 79.9            |
| Accra        | -24.8           | 26.4             | 76.3            |
| Addis Ababa  | -31.8           | 16.0             | 63.9            |
| ...          | ...             | ...              | ...             |
| Zagreb       | -39.2           | 10.7             | 58.1            |
| Zanzibar City| -26.5           | 26.0             | 75.2            |
| Zürich       | -42.0           | 9.3              | 63.6            |
| Ürümqi       | -42.1           | 7.4              | 56.7            |
| İzmir        | -34.4           | 17.9             | 67.9            |



## Sobre:

Abaixo um índice como todas as etapas para resolução do projeto, desde informações sobre as configurações desktop, sobre a memória, ssd, como gerar os dados, dependências e o resultado do tempo de execução de cada script utilizados.

## índice

<a id="voltar"></a>

1.  **[Decoradores](#ancora01)**
2.  **[Scripts para gerar os dados.](#ancora02)**
3.  **[Pandas - min, max e mean em 1 bilhão de linhas](#ancora03)**
4.  **[Exercício 3: Filtragem de Logs por Severidade](#ancora04)**
5.  **[Exercício 4: Validação de Dados de Entrada](#ancora05)**
6.  **[Exercício 5: Detecção de Anomalias em Dados de Transações](#ancora05)**
7.  **[Exercício 6: Contagem de Palavras em Textos](#ancora06)**
8.  **[Exercício 7: Normalização de Dados](#ancora07)**
9.  **[Exercício 8: Filtragem de Dados Faltantes](#ancora08)**
10.  **[Exercício 9: Extração de Subconjuntos de Dados](#ancora09)**
11. **[Exercício 10: Agregação de Dados por Categoria](#ancora10)**
12. **[Exercício 11:  Leitura de Dados até Flag](#ancora11)**
13. **[Exercício 12: Validação de Entrada](#ancora12)**
14. **[Exercício 13: Consumo de API Simulado](#ancora13)**
15. **[Exercício 14: Tentativas de Conexão](#ancora14)**
16. **[Exercício 15: Processamento de Dados com Condição de Parada](#ancora15)**
17. **[Desafio : Estruturas de Controle de Fluxo](#desafio)**


<a id="ancora01"></a>
## Decoradores

**O decorator timer serve para medir o tempo de execução de uma função e registrar o resultado em um arquivo CSV. Ele também imprime o tempo de execução no console.**

**Porquê usar decorators?**
* Nesse exemplo precisamos medir o tempo de cada uma das funções que serão criadas para determinar a melhor performance, dessa forma podemos criar uma função mais limpa e organizada onde iremos chamar o decorator para exibir e registrar as informações do tempo de execução da cada uma das funções, isso evita a duplicidade de código.

* Funcionalidades:

    - Mede o tempo de execução da função decorada.
    - Formata o tempo em horas, minutos e segundos.
    - Registra o nome da função e o tempo de execução em um arquivo CSV chamado tempos_execucao.csv.
    - Imprime o nome da função e o tempo de execução no console.
* Vantagens de usar decorators:

    - Códigos mais concisos e reutilizáveis.
    - Evita a duplicação de código para medição de tempo.
    - Facilita a comparação do tempo de execução de diferentes funções.
    - Permite a criação de logs de performance.

* **Decorator para salvar os resultado em um csv**.

In [64]:
%%writefile src/utils/decorators.py 
#decorator para registrar o tempo de execução da função em um .csv
import time
import csv
import threading  # Importe o módulo threading aqui

def timer_to_csv(func):
    def format_time(segundos: int): 
        """
        Formata os milisegundos em hora:minuto:segundo
        """
        if segundos < 60:
            return f"{segundos:.3f} segundos"
        elif segundos < 3600:
            minutos, segundos = divmod(segundos, 60)
            return f"{int(minutos)} minutos {int(segundos)} segundos"
        else:
            horas, remainder = divmod(segundos, 3600)
            minutos, segundos = divmod(remainder, 60)
            if minutos == 0:
                return f"{int(horas)} horas {int(segundos)} segundos"
            else:
                return f"{int(horas)} horas {int(minutos)} minutos {int(segundos)} segundos"
    
    def print_elapsed_time(start_time, finished_flag):
        while not finished_flag.is_set():
            elapsed_time = time.time() - start_time
            print(f"Tempo decorrido: {format_time(elapsed_time)}", end="\r")
            time.sleep(1)  # Atualiza a cada segundo

    def wrapper(*args, **kwargs):
        start_time = time.time()
        
        # Inicializa um sinalizador booleano para indicar se a função terminou
        finished_flag = threading.Event()
        
        # Iniciar uma thread para imprimir o tempo decorrido em tempo real
        thread = threading.Thread(target=print_elapsed_time, args=(start_time, finished_flag))
        thread.start()
        
        result = func(*args, **kwargs)
        elapsed_time = time.time() - start_time
        print(f"Tempo total de execução: {format_time(elapsed_time)}")
        
        # Sinaliza que a função terminou
        finished_flag.set()
        
        # Salvando o nome da função e o tempo de execução em um arquivo CSV
        with open('data/tempos_execucao.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([func.__name__, format_time(elapsed_time)])
        
        return result    
    return wrapper


Overwriting src/utils/decorators.py


[voltar](#voltar)

<a id="ancora02"></a>
## Scripts para gerar os dados.

### Script utilizando Pandas - not suport 1billion rows

In [2]:
# Instalação da lib Pandas;
!poetry add pandas -q

In [52]:
%%time
import pandas as pd
import random

def generate_data(num_rows):
    """
    Gera dados aleatórios de temperatura para cidades e salva em um arquivo CSV.

    Args:
        num_rows (int): Número de linhas desejadas para o DataFrame.

    Returns:
        None
    """
    # Amostrar aleatoriamente os nomes das cidades existentes
    city_names = pd.read_csv('data/weather_stations.csv', sep=';', header=None, skiprows=2, usecols=[0]).sample(num_rows, replace=True).iloc[:,0].tolist()

    # Gerar temperaturas aleatórias
    temperatures = [round(random.uniform(-89.2, 56.7), 1) for _ in range(num_rows)]

    # Criar um DataFrame pandas com os dados gerados
    df = pd.DataFrame({'city': city_names, 'temperature': temperatures})
    
    # Salvar o DataFrame em um arquivo CSV
    df.to_csv('data/measurements_pandas.txt', sep=';', header=False, index=False)

    # salvar o valor da quantidade de linhas solicitada   
    with open('data/num_rows.txt', 'w') as file:
        # Escrever o valor no arquivo
        file.write(str(num_rows))

if __name__ == "__main__":
    num_rows = int(input("Digite a quantidade de linhas desejadas: "))
    generate_data(num_rows)


Digite a quantidade de linhas desejadas:  1000000


CPU times: user 3.06 s, sys: 289 ms, total: 3.35 s
Wall time: 16.3 s


### Scrip python para gerar dados aleatórios
* **O script foi retirado do desafio [The One Billion Row Challenge](https://github.com/gunnarmorling/1brc), originalmente proposto para Java.**
* **Algumas alterações foram realizadas, como por exemplo a temp máxima e mínima foram alterados para valores históricos reais.**

In [8]:
%%time
# Script para gerar 1 bilhão de linhas com dados aleatórios.
# Based on https://github.com/gunnarmorling/1brc/blob/main/src/main/java/dev/morling/onebrc/Createtempments.java

import os
import sys
import random
import time


def check_args(file_args):
    """
    Sanity checks out input and prints out usage if input is not na positive integer
    """
    try:
        if len(file_args) != 2 or int(file_args[1]) <= 0:
            raise Exception()
    except:
        print("Usage:  create_tempments.sh <positive integer number of records to create>")
        print("        You can use underscore notation for large number of records.")
        print("        For example:  1_000_000_000 for one billion")
        exit()


def build_weather_city_name_list():
    """
    Grabs the weather city names from example data provided in repo and dedups
    
    """   
    city_names = []
    with open('data/weather_stations.csv', 'r') as file:

        file_contents = file.read()
    for city in file_contents.splitlines():
        if "#" in city:
            next
        else:
            city_names.append(city.split(';')[0])
    return list(set(city_names))


def convert_bytes(num):
    """
    Convert bytes to a human-readable format (e.g., KiB, MiB, GiB)
    """
    for x in ['bytes', 'KiB', 'MiB', 'GiB']:
        if num < 1024.0:
            return "%3.1f %s" % (num, x)
        num /= 1024.0


def format_elapsed_time(seconds):
    """
    Format elapsed time in a human-readable format
    """
    if seconds < 60:
        return f"{seconds:.3f} seconds"
    elif seconds < 3600:
        minutes, seconds = divmod(seconds, 60)
        return f"{int(minutes)} minutes {int(seconds)} seconds"
    else:
        hours, remainder = divmod(seconds, 3600)
        minutes, seconds = divmod(remainder, 60)
        if minutes == 0:
            return f"{int(hours)} hours {int(seconds)} seconds"
        else:
            return f"{int(hours)} hours {int(minutes)} minutes {int(seconds)} seconds"


def estimate_file_size(weather_city_names, num_rows_to_create):
    """
    Tries to estimate how large a file the test data will be
    """
    total_name_bytes = sum(len(s.encode("utf-8")) for s in weather_city_names)
    avg_name_bytes = total_name_bytes / float(len(weather_city_names))

    # avg_temp_bytes = sum(len(str(n / 10.0)) for n in range(-999, 1000)) / 1999
    avg_temp_bytes = 4.400200100050025

    # add 2 for separator and newline
    avg_line_length = avg_name_bytes + avg_temp_bytes + 2

    human_file_size = convert_bytes(num_rows_to_create * avg_line_length)

    return f"Estimated max file size is:  {human_file_size}."


def build_test_data(weather_city_names, num_rows_to_create):
    """
    Generates and writes to file the requested length of test data
    """
    start_time = time.time()
    coldest_temp = -89.2
    hottest_temp = 56.7
    city_names_10k_max = random.choices(weather_city_names, k=10_000)
    batch_size = 10000 # instead of writing line by line to file, process a batch of citys and put it to disk
    chunks = num_rows_to_create // batch_size
    print('Building test data...')

    try:
        with open("data/measurements.txt", 'w') as file:
            progress = 0
            for chunk in range(chunks):
                
                batch = random.choices(city_names_10k_max, k=batch_size)
                prepped_deviated_batch = '\n'.join([f"{city};{random.uniform(coldest_temp, hottest_temp):.1f}" for city in batch]) # :.1f should quicker than round on a large scale, because round utilizes mathematical operation
                file.write(prepped_deviated_batch + '\n')

                
                # Update progress bar every 1%
                if (chunk + 1) * 100 // chunks != progress:
                    progress = (chunk + 1) * 100 // chunks
                    bars = '=' * (progress // 2)
                    sys.stdout.write(f"\r[{bars:<50}] {progress}%")
                    sys.stdout.flush()
        sys.stdout.write('\n')
    except Exception as e:
        print("Something went wrong. Printing error info and exiting...")
        print(e)
        exit()
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    file_size = os.path.getsize("data/measurements.txt")
    human_file_size = convert_bytes(file_size)
 
    print("Test data successfully written to data/measurements.txt")
    print(f"Actual file size:  {human_file_size}")
    print(f"Elapsed time: {format_elapsed_time(elapsed_time)}")


def main():
    """
    main program function
    """
    num_rows_to_create = 500_000_000
    weather_city_names = []
    weather_city_names = build_weather_city_name_list()
    print(estimate_file_size(weather_city_names, num_rows_to_create))
    build_test_data(weather_city_names, num_rows_to_create)
    print("Test data build complete.")


if __name__ == "__main__":
    main()
exit()

Estimated max file size is:  7.4 GiB.
Building test data...
[==================================================] 100%
Test data successfully written to data/measurements1.txt
Actual file size:  7.4 GiB
Elapsed time: 7 minutes 45 seconds
Test data build complete.
CPU times: user 7min 34s, sys: 11.1 s, total: 7min 45s
Wall time: 7min 45s


#### Como executar o script:
$`cd aula_05/src/` #para acessar a pasta dos scripts. 

$`python data_generate.py`#para criar os dados na pasta 'data/mesurements.txt'

* **obs**: O script depende do arquivo 'data/weather_stations.csv' para gerar os dados das cidades.



* **Agora com os dados de 1 bilhao gerados vamos ver quem se sai melhor para as configuracoes da minha maquina**.

<a id="ancora03"></a>
## Pandas - min, max e mean em 1 bilhão de linhas

In [42]:
# Instalando a lib pandas.
!poetry add pandas tabulate -q

### Documentação do Script Python: Calculando Mínimo, Máximo e Média para um Bilhão de Linhas

Este script calcula a temperatura mínima, máxima e média para cada cidade em um grande conjunto de dados contendo um bilhão de linhas. Ele utiliza as bibliotecas Pandas, multiprocessing e tqdm.

**Funcionalidade:**

1. **Leitura de dados:** 
    * Lê dados de um arquivo CSV (`data/measurements.txt`) com colunas nomeadas `cidade` e `temp`.
    * Emprega fragmentação para processar os dados em partes menores e gerenciáveis.

2. **Processamento paralelo:**
    * Aproveita o multiprocessing para distribuir a carga de trabalho entre os núcleos de CPU disponíveis.
    * Cada fragmento é processado independentemente pela função `process_chunk`.

3. **Agregação:**
    * Dentro de cada fragmento, a função `process_chunk` usa Pandas para:
        * Agrupar dados por `cidade`.
        * Calcular o `max`, `min` e `mean` da coluna `temp`.
        * Redefinir o índice para obter um DataFrame limpo.

4. **Agregação de resultados:**
    * Resultados de fragmentos individuais são concatenados em um único DataFrame.
    * Uma agregação final é realizada para calcular o `max`, `min` e `mean` geral para cada cidade.
    * O DataFrame final é classificado por `cidade`.

5. **Visualização do progresso:**
    * A biblioteca tqdm fornece uma barra de progresso para visualizar o processamento dos fragmentos.

**Principais características:**

* **Processamento eficiente de grandes conjuntos de dados:** A fragmentação e o multiprocessing permitem lidar com dados massivos com eficiência.
* **Desempenho otimizado:** A utilização de vários núcleos reduz significativamente o tempo de processamento.

**Observações:**

* Certifique-se de que a variável `filename` aponte para o local correto do arquivo de dados `data/measurements.txt`.
* Ajuste `chunksize` dependendo da memória do seu sistema e dos recursos de processamento.

**Documentação**: [Pandas](https://pandas.pydata.org/docs/)

**Como executar o script:**

1. **Terminal Bash digite:** `cd aula_05`
2. **Execute o script:** Digite o seguinte comando e pressione Enter: 
```bash
python src/pandas_df.py


In [1]:
!python src/pandas_df.py

Iniciando o processamento do arquivo.
Tempo total de execução: 1.379 segundos
                   city   max   min   mean
0              A Coruña  52.7 -88.1 -17.34
1              A Yun Pa  50.6 -85.3 -21.25
2              Aabenraa  47.7 -88.5 -37.76
3                Aachen  38.8 -77.3 -33.72
4                Aadorf  50.2 -85.2 -13.87
...                 ...   ...   ...    ...
41338   ’Tlat Bni Oukil  48.1 -81.7 -21.87
41339     ’s-Gravendeel  54.0 -75.7  -7.58
41340    ’s-Gravenzande  23.4 -82.3 -13.29
41341     ’s-Heerenberg  55.0 -88.7 -19.00
41342  ’s-Hertogenbosch  54.2 -73.9  -1.58

[41343 rows x 4 columns]


### Script Pandas_df:

In [ ]:
%%writefile src/pandas_df.py
# Pandas=> script para caluclar min, max e mean em um bilhão de linhas.
import pandas as pd 
import time
from multiprocessing import Pool, cpu_count
from utils.decorators import timer_to_csv  # Importa o decorador
#from tabulate import tabulate

CONCURRENCY = cpu_count()

filename = "data/measurements_pandas.txt"  # Certifique-se de que este é o caminho correto para o arquivo

def get_total_lines(num_rows_path):
    with open(num_rows_path, 'r') as f:
        total_lines = f.read()
    return int(total_lines)

def process_chunk(chunk):
    # Agrega os dados dentro do chunk usando Pandas
    aggregated = chunk.groupby('city')['temp'].agg(['max','min','mean']).reset_index()
    return aggregated

@timer_to_csv  # Aplica o decorador
def create_df_with_pandas(filename, num_rows_path):
    total_linhas = get_total_lines(num_rows_path)
    chunksize = total_linhas // 10 + (1 if total_linhas % 10 else 0)  # Define o tamanho do chunk
    with pd.read_csv(filename, sep=';', header=None, names=['city', 'temp'], chunksize=chunksize) as reader:
        with Pool(CONCURRENCY) as pool:
            results = []
            for result in pool.imap(process_chunk, reader):
                results.append(result)

    final_df = pd.concat(results, ignore_index=True)

    final_aggregated_df = final_df.groupby('city').agg({
        'max': 'max',
        'min': 'min',
        'mean': 'mean'
        
    }).reset_index().round(2).sort_values('city')

    return final_aggregated_df

if __name__ == "__main__":    
    print("Iniciando o processamento do arquivo.")
    num_rows_path= "data/num_rows.txt"
    filename = "data/measurements_pandas.txt"     
    df = create_df_with_pandas(filename, num_rows_path)
    #print(tabulate(df, headers='keys', tablefmt='pretty'))
    print(df)

## Polars - min, max e mean em 1 bilhão de linhas

In [11]:
# Instalando a lib polars
!poetry add polars -q

### Documentação do Script `polars_df.py`

Este script Python utiliza a biblioteca Polars para calcular de forma eficiente a temperatura mínima, máxima e média para cada cidade em um conjunto de dados grande (por exemplo, um bilhão de linhas). 

**Funcionalidade:**

1. **Importações:**
    * Importa a função `timer_to_csv` do módulo `utils.decorators`. 
    * Importa a biblioteca `polars` como `pl`.

2. **Configuração do Polars:**
    * Define o tamanho do chunk para processamento de streaming como 100.000 linhas usando `pl.Config.set_streaming_chunk_size(100000)`. Isso otimiza o uso de memória ao lidar com grandes conjuntos de dados.

3. **Função `polars_df()`:**
    * Esta função é decorada com `@timer_to_csv`, que mede o tempo de execução e salva os resultados em um arquivo CSV.
    * Lê o arquivo CSV `data/measurements.txt` com as seguintes características:
        * Separador: `;`
        * Sem cabeçalho (`has_header=False`)
        * Esquema de colunas: `{"city": pl.String, "temp": pl.Float64}`
    * Agrupa os dados por `city`.
    * Calcula as seguintes estatísticas para cada cidade:
        * Temperatura máxima (`max_temp`)
        * Temperatura mínima (`min_temp`)
        * Temperatura média (`mean_temp`)
    * Ordena os resultados por `city`.
    * Coleta os resultados em um DataFrame Polars usando `collect(streaming=True)` para processamento eficiente de grandes conjuntos de dados.
    * Retorna o DataFrame Polars resultante.

4. **Execução principal:**
    * Se o script for executado diretamente (e não importado como um módulo), a função `polars_df()` é chamada e o DataFrame resultante é impresso no console.

**Principais características:**

* **Processamento eficiente de grandes conjuntos de dados:** O Polars é otimizado para lidar com grandes conjuntos de dados, utilizando processamento em chunks e técnicas eficientes de gerenciamento de memória.
* **Sintaxe expressiva:** A API do Polars permite realizar a análise de dados de forma concisa e legível.
* **Desempenho:** O Polars é frequentemente mais rápido que o Pandas, especialmente em grandes conjuntos de dados.

**Documentação:** [Polars](https://docs.pola.rs/#philosophy)

Este script demonstra como o Polars pode ser usado para processar grandes conjuntos de dados de forma eficiente e calcular estatísticas básicas. 

**Como executar o script:**

1. **Terminal Bash digite:** `cd aula_05`
2. **Execute o script:** Digite o seguinte comando e pressione Enter: 
```bash
python src/polars_df.py

In [2]:
!python src/polars_df.py

Tempo total de execução: 0.272 segundos
shape: (41_343, 4)
┌──────────────────┬──────────┬──────────┬────────────┐
│ city             ┆ max_temp ┆ min_temp ┆ mean_temp  │
│ ---              ┆ ---      ┆ ---      ┆ ---        │
│ str              ┆ f64      ┆ f64      ┆ f64        │
╞══════════════════╪══════════╪══════════╪════════════╡
│ A Coruña         ┆ 52.7     ┆ -88.1    ┆ -13.647368 │
│ A Yun Pa         ┆ 50.6     ┆ -85.3    ┆ -21.007407 │
│ Aabenraa         ┆ 47.7     ┆ -88.5    ┆ -28.036842 │
│ Aachen           ┆ 38.8     ┆ -77.3    ┆ -30.217391 │
│ Aadorf           ┆ 50.2     ┆ -85.2    ┆ -13.6      │
│ …                ┆ …        ┆ …        ┆ …          │
│ ’Tlat Bni Oukil  ┆ 48.1     ┆ -81.7    ┆ -19.422222 │
│ ’s-Gravendeel    ┆ 54.0     ┆ -75.7    ┆ -6.096429  │
│ ’s-Gravenzande   ┆ 23.4     ┆ -82.3    ┆ -22.825    │
│ ’s-Heerenberg    ┆ 55.0     ┆ -88.7    ┆ -5.705556  │
│ ’s-Hertogenbosch ┆ 54.2     ┆ -73.9    ┆ -5.0       │
└──────────────────┴──────────┴──────────┴───

### Script Polars_df.py

In [12]:
%%writefile src/polars_df.py
# Polars => script para caluclar min, max e mean em um bilhão de linhas.
from utils.decorators import timer_to_csv
import polars as pl

def get_total_lines(num_rows_path):
    with open(num_rows_path, 'r') as f:
        total_lines = f.read()
    return int(total_lines)

@timer_to_csv
def polars_df(filename, num_rows_path):
    total_linhas = get_total_lines(num_rows_path)
    chunksize = total_linhas // 10 + (1 if total_linhas % 10 else 0)  # Define o tamanho do chunk

    pl.Config.set_streaming_chunk_size(chunksize)

    # Leitura do arquivo CSV e definição do schema
    return (pl.scan_csv(filename, separator=";", has_header=False,
                        schema={"city": pl.String, "temp": pl.Float64})
                        .group_by("city").agg(
                                                 max_temp=pl.col("temp").max(),
                                                 min_temp=pl.col("temp").min(),
                                                 mean_temp=pl.col("temp").mean()
                                                ).sort("city").collect(streaming=True)
           )

if __name__ == "__main__":  
    num_rows_path= "data/num_rows.txt"
    filename = "data/measurements_pandas.txt"  # Certifique-se de que este é o caminho correto para o arquivo
    df = polars_df(filename, num_rows_path)
    print(df)

Overwriting src/polars_df.py


## Duckdb - min, max e mean em 1 bilhão de linhas

In [14]:
!poetry add duckdb -q

### Documentação do Script `duckdb_df.py`

Este script Python utiliza a biblioteca DuckDB para calcular de forma eficiente a temperatura mínima, máxima e média para cada cidade em um conjunto de dados grande (por exemplo, um bilhão de linhas). 

**Funcionalidade:**

1. **Importações:**
    * Importa a função `timer_to_csv` do módulo `utils.decorators`. 
    * Importa a biblioteca `duckdb`.

2. **Função `duckdb_df()`:**
    * Esta função é decorada com `@timer_to_csv`, que mede o tempo de execução e salva os resultados em um arquivo CSV.
    * Executa uma consulta SQL no DuckDB para:
        * Ler o arquivo CSV `data/measurements.txt` com as seguintes características:
            * Detecção automática de tipo de dados desativada (`AUTO_DETECT=FALSE`)
            * Separador: `;`
            * Colunas: `city` (VARCHAR) e `temp` (DECIMAL)
        * Agrupar os dados por `city`.
        * Calcular as seguintes estatísticas para cada cidade:
            * Temperatura máxima (`max_temp`)
            * Temperatura mínima (`min_temp`)
            * Temperatura média (`mean_temp`) - convertida para DECIMAL
        * Ordenar os resultados por `city`.
    * Exibe os resultados da consulta no console.

3. **Execução principal:**
    * Se o script for executado diretamente (e não importado como um módulo), a função `duckdb_df()` é chamada.

**Documentação:** [Duckdb](https://duckdb.org/docs/)

**Como executar o script:**

1. **Terminal Bash digite:** `cd aula_05`
2. **Execute o script:** Digite o seguinte comando e pressione Enter: 
```bash
python src/duckdb_df.py

In [3]:
!python src/duckdb_df.py

                   city  max_temp  min_temp  mean_temp
0              A Coruña      52.7     -88.1    -13.647
1              A Yun Pa      50.6     -85.3    -21.007
2              Aabenraa      47.7     -88.5    -28.037
3                Aachen      38.8     -77.3    -30.217
4                Aadorf      50.2     -85.2    -13.600
...                 ...       ...       ...        ...
41338   ’Tlat Bni Oukil      48.1     -81.7    -19.422
41339     ’s-Gravendeel      54.0     -75.7     -6.096
41340    ’s-Gravenzande      23.4     -82.3    -22.825
41341     ’s-Heerenberg      55.0     -88.7     -5.706
41342  ’s-Hertogenbosch      54.2     -73.9     -5.000

[41343 rows x 4 columns]
Tempo total de execução: 0.873 segundos


### Script Duck_db:

In [6]:
%%writefile src/duckdb_df.py
# Duckdb => script para caluclar min, max e mean em um bilhão de linhas.
from utils.decorators import timer_to_csv
import duckdb

def get_total_lines(num_rows_path):
    with open(num_rows_path, 'r') as f:
        total_lines = f.read()
    return int(total_lines)

@timer_to_csv
def create_duckdb(filename,num_rows_path):
    total_linhas = get_total_lines(num_rows_path)
    chunksize = total_linhas // 10 + (1 if total_linhas % 10 else 0)  # Define o tamanho do chunk
    conn = duckdb.connect(':memory:')
    query = f"""
            SELECT city,
                MAX(temp) AS max_temp,
                MIN(temp) AS min_temp,
                CAST(AVG(temp) AS DECIMAL()) AS mean_temp                
            FROM read_csv("{filename}", AUTO_DETECT=FALSE, sep=';', columns={{'city':VARCHAR, 'temp': 'DECIMAL'}})
            GROUP BY city
            ORDER BY city
        """
    print(conn.execute(query).df())

if __name__ == "__main__":
    filename = "data/measurements_pandas.txt" 
    num_rows_path= "data/num_rows.txt"
    create_duckdb(filename,num_rows_path) 

Overwriting src/duckdb_df.py


## Dask - min, max e mean em 1 bilhão de linhas

In [160]:
#necessário Instalar.
!poetry add dask-expr -q

In [127]:
!poetry remove dask-expr -q

### Documentação do Script Dask:

**Descrição:**

Este script demonstra como calcular a temperatura mínima, máxima e média para cada cidade em um conjunto de dados massivo contendo um bilhão de linhas. Ele utiliza as bibliotecas Dask para otimizar o processamento e fornecer uma experiência interativa.

**Funcionalidade:**

**1. Leitura de dados:**

* Lê dados de um arquivo CSV (`data/measurements.txt`) com colunas nomeadas `city` e `temp`.
* Emprega a biblioteca Dask para ler o arquivo em um DataFrame particionado, permitindo processamento eficiente em grandes conjuntos de dados.

**2. Processamento paralelo:**

* O Dask gerencia o particionamento e o processamento paralelo do DataFrame em diferentes threads ou processos.
* Cada partição é processada independentemente pela função `process_chunk`.

**3. Agregação:**

* Dentro de cada partição, a função `process_chunk` usa Pandas para:
    * Agrupar dados por `city`.
    * Calcular o `max`, `min` e `mean` da coluna `temp`.
    * Redefinir o índice para obter um DataFrame limpo.

**4. Agregação de resultados:**

* Os resultados de cada partição são combinados em um único DataFrame.
* Uma agregação final é realizada para calcular o `max`, `min` e `mean` geral para cada cidade.
* O DataFrame final é ordenado por `city`.

**Características:**

* **Processamento eficiente de grandes conjuntos de dados:** O Dask permite lidar com dados massivos de forma eficiente e escalável.
* **Desempenho otimizado:** O processamento paralelo em partições reduz significativamente o tempo de processamento.

**Observações:**

* Certifique-se de que a variável `filename` aponte para o local correto do arquivo de dados `data/measurements.txt`.
* Ajuste o tamanho da partição (`chunksize`) de acordo com a memória do seu sistema e os recursos de processamento.
* Se necessário, configure o Dask para usar um cluster de computação para aumentar ainda mais o desempenho.

**Documentação:** [Dask](https://docs.dask.org/en/stable/)

**Execução:**

1. **Navegue até o diretório do script:** `cd aula_05`
2. **Execute o script:** `python src/dask_df.py`


In [4]:
!python src/dask_df.py

                  temp             
                   max   min   mean
city                               
A Coruña          52.7 -88.1 -13.65
A Yun Pa          50.6 -85.3 -21.01
Aabenraa          47.7 -88.5 -28.04
Aachen            38.8 -77.3 -30.22
Aadorf            50.2 -85.2 -13.60
...                ...   ...    ...
’Tlat Bni Oukil   48.1 -81.7 -19.42
’s-Gravendeel     54.0 -75.7  -6.10
’s-Gravenzande    23.4 -82.3 -22.82
’s-Heerenberg     55.0 -88.7  -5.71
’s-Hertogenbosch  54.2 -73.9  -5.00

[41343 rows x 3 columns]
Tempo total de execução: 0.795 segundos


### Script Dask_df:

In [9]:
%%writefile src/dask_df.py
# Dask => script para caluclar min, max e mean em um bilhão de linhas.
from utils.decorators import timer_to_csv
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd
@timer_to_csv
def dask_df(filename):    
    # Ler o arquivo txt diretamente em um DataFrame Dask
    df = dd.read_csv(filename, delimiter=';', 
                     header=None, names=['city', 'temp'])
    # min, max, e mean pela cidade ordenado pelo index
    return print(df.groupby('city').
                   agg({'temp': ['max','min','mean']}).
                   compute().round(2).sort_index())

if __name__ == "__main__":
    filename = "data/measurements_pandas.txt"
    dask_df(filename)

Writing src/dask_df.py


## Pyspark - min, max e mean em 1 bilhão de linhas.

In [ ]:
#instalação da lib
!poetry add pyspark -q

### Documentação do Script PySpark:

**Descrição:**

Este script demonstra como calcular a temperatura mínima, máxima e média para cada cidade em um conjunto de dados massivo contendo um bilhão de linhas utilizando a biblioteca PySpark.

**Funcionalidade:**

**1. Inicialização da sessão Spark:**

* Cria um objeto `SparkSession` para interagir com o cluster Spark.
* Define o nome da aplicação como "Temperature Analysis".

**2. Leitura de dados:**

* Lê o arquivo CSV `data/measurements.txt` diretamente em um DataFrame Spark.
* Especifica que o arquivo não possui cabeçalho e usa ponto-e-vírgula como delimitador.
* Define os nomes das colunas como "City" e "Temperature".

**3. Conversão de tipo:**

* Converte a coluna "Temperature" para o tipo numérico `float` para possibilitar cálculos.

**4. Cálculo de estatísticas:**

* Agrupa o DataFrame por "City".
* Calcula o máximo, mínimo e média da temperatura para cada cidade utilizando funções Spark SQL.
* Arredonda o valor médio da temperatura para duas casas decimais.

**5. Ordenação:**

* Ordena as estatísticas pela coluna "City".

**6. Exibição de resultados:**

* Imprime o DataFrame com as estatísticas na tela.

**7. Encerramento da sessão Spark:**

* Libera recursos utilizados pelo Spark.

**Características:**

* **Processamento distribuído:** PySpark distribui o processamento por múltiplos nós em um cluster, permitindo lidar com conjuntos de dados massivos de maneira eficiente.
* **Otimização para grandes dados:** PySpark oferece otimizações específicas para manipular grandes volumes de dados.
* **Linguagem SQL familiar:** Usa Spark SQL para realizar consultas e transformações nos dados, facilitando a utilização para quem conhece SQL.

**Observações:**

* Certifique-se de ter o PySpark instalado e configurado corretamente.
* O arquivo `data/measurements.txt` deve existir no caminho especificado.
* A função `timer_to_csv` salva o tempo de execução em um arquivo CSV.

**Documentação:** [pyspark](https://spark.apache.org/docs/latest/api/python/index.html)

**Execução:**

1. **Navegue até o diretório do script:** `cd aula_05`
2. **Execute o script:** `python src/pyspark_df.py`

In [5]:
!python src/pyspark_df.py

24/03/26 23:58:59 WARN Utils: Your hostname, DESKTOP-AN64GAS resolves to a loopback address: 127.0.1.1; using 172.25.237.139 instead (on interface eth0)
24/03/26 23:58:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/26 23:59:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
+----------+---------------+---------------+---------------+                    
|      City|Max Temperature|Min Temperature|Avg Temperature|
+----------+---------------+---------------+---------------+
|  A Coruña|           52.7|          -88.1|         -13.65|
|  A Yun Pa|           50.6|          -85.3|         -21.01|
|  Aabenraa|           47.7|          -88.5|         -28.04|
|    Aachen|           38.8|          -77.3|         -30.22|
|    Aadorf|           50.2|          -

### Script pyspark_df:

In [13]:
%%writefile src/pyspark_df.py
# Pyspark => script para caluclar min, max e mean em um bilhão de linhas.
from utils.decorators import timer_to_csv
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, min as spark_min, max as spark_max, avg as spark_avg, round as spark_round

@timer_to_csv
def pyspark_df(filename):     
    # Inicializar uma sessão Spark
    spark = SparkSession.builder \
        .appName("Temperature Analysis") \
        .getOrCreate()
    
    # Ler o arquivo CSV diretamente em um DataFrame Spark
    df = spark.read.option("header", "false").option("delimiter", ";").csv(filename) \
        .toDF("City", "Temperature")
    
    # Converter a coluna 'Temperature' para tipo numérico
    df = df.withColumn("Temperature", col("Temperature").cast("float"))
    
    # Calcular estatísticas usando Spark SQL
    statistics = df.groupBy("City") \
        .agg(spark_max("Temperature").alias("Max Temperature"),
             spark_min("Temperature").alias("Min Temperature"),             
             spark_round(spark_avg("Temperature"),2).alias("Avg Temperature"))
    
    # Ordenar as estatísticas pela cidade
    statistics_sorted = statistics.orderBy("City")
    
    # Mostrar as estatísticas
    return statistics_sorted.show()
    
    # Encerrar a sessão Spark
    spark.stop()

if __name__ == "__main__":
    filename = "data/measurements_pandas.txt"
    pyspark_df(filename) 


Writing src/pyspark_df.py


## Vaex - min, max e mean em 1 bilhão de linhas.

In [68]:
!poetry add vaex -q

### Documentação do Script Vaex:

**Descrição:**

Este script demonstra como calcular a temperatura mínima, máxima e média para cada cidade em um grande conjunto de dados, possivelmente contendo um bilhão de linhas, utilizando a biblioteca Vaex. O Vaex é uma biblioteca Python especializada no processamento eficiente de grandes tabelas de dados.

**Funcionalidade:**

**1. Leitura de dados:**

* Lê o arquivo CSV `data/measurements_pandas.txt` utilizando a função `vaex.from_csv`.
* Define os nomes das colunas como "city" e "temp" e o separador como ponto-e-vírgula (;).

**2. Cálculo de estatísticas:**

* Agrupa o DataFrame por "city" usando `df.groupby(df['city'])`.
* Calcula o máximo, mínimo e média da temperatura para cada cidade utilizando a função `agg` com uma lista de funções de agregação.
* Arredonda o valor médio da temperatura para duas casas decimais usando o método `round(2)`.

**3. Exibição de resultados:**

* A função `print` exibe o DataFrame final contendo as estatísticas na tela.

**Características:**

* **Processamento eficiente de grandes dados:** O Vaex é otimizado para lidar com grandes volumes de dados na memória, permitindo cálculos rápidos em tabelas com bilhões de linhas.
* **Operações em memória:** Realiza a maioria das operações na memória principal, evitando acesso frequente ao disco e melhorando a performance.
* **Uso intuitivo:** Oferece uma interface similar ao Pandas, facilitando a migração de scripts existentes.

**Execução:**

1. **Navegue até o diretório do script:** `cd src`
2. **Execute o script:** `python vaex_df.py`


In [6]:
!python src/vaex_df.py

#       city              temp_max    temp_min    temp_mean
0       A Coruña          52.7        -88.1       -13.64736842105263
1       A Yun Pa          50.6        -85.3       -21.007407407407406
2       Aabenraa          47.7        -88.5       -28.036842105263155
3       Aachen            38.8        -77.3       -30.21739130434783
4       Aadorf            50.2        -85.2       -13.6
...     ...               ...         ...         ...
41,338  ’Tlat Bni Oukil   48.1        -81.7       -19.42222222222222
41,339  ’s-Gravendeel     54.0        -75.7       -6.096428571428571
41,340  ’s-Gravenzande    23.4        -82.3       -22.825
41,341  ’s-Heerenberg     55.0        -88.7       -5.705555555555555
41,342  ’s-Hertogenbosch  54.2        -73.9       -5.0
Tempo total de execução: 1.045 segundos


### Script vaex_df:

In [70]:
%%writefile src/vaex_df.py
# Vaex => script para caluclar min, max e mean.
from utils.decorators import timer_to_csv
import vaex
@timer_to_csv
def vaex_df(filename):
    # Leitura do arquivo CSV utilizando Vaex
    df = vaex.from_csv(filename, sep=';', header=None, names=['city', 'temp'])

    # Cálculo das estatísticas
    combined_results = df.groupby(df['city']).agg({'temp': ['max', 'min', 'mean']})

    # Ordenar por 'city'
    combined_results = combined_results.sort(by='city')
    
    # Exibição dos resultados
    return print(combined_results)
   
if __name__ == "__main__":
    filename = "data/measurements_pandas.txt"
    create_vaex(filename)


Writing src/vaex_df.py


## cudf com pandas via GPU - min, max e mean

### Para utilizar precisa ter uma GPU compatível e se possui DriveToolKit instalado conforme o link da [Nvidia](https://developer.nvidia.com/cuda-gpus#compute).

* **Confira a versão cuda instalado com o comando abaixo:**

In [7]:
!nvidia-smi

Tue Mar 26 23:59:39 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.40.06              Driver Version: 551.23         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 Ti     On  |   00000000:03:00.0  On |                  N/A |
|  0%   43C    P8             15W /  200W |     668MiB /   8192MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

* **No meu caso estou com a versão 12+ com essa informação acesse o site para escolher a versão correta no site:**
[RAPIDS-Nvidi](https://docs.rapids.ai/install)

In [8]:
#verificando a versão do python para esse estudo.
!python --version

Python 3.10.13


In [9]:
#Necessário fazer upgrade do pip
!pip install -U pip

In [3]:
# Instalação somente pelo pip. não funciona com poetry add.
!pip install --no-cache-dir --extra-index-url https://pypi.nvidia.com cudf-cu12 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-expr 1.0.5 requires pandas>=2, but you have pandas 1.5.3 which is incompatible.


#### Após a instalação é necessário reiniciar o kernel do jupyter:


In [4]:
#Reinicia o kernel jupyter
get_ipython().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

In [2]:
# CuDF_Pandas=> script para caluclar min, max e mean em um bilhão de linhas na GPU rtx 3060ti.
# Necessário possuir uma GPU compativel e instalar o pacote RAPIDS da Nvidia.
%load_ext cudf.pandas
import pandas as pd
from src.utils.decorators import timer_to_csv
@timer_to_csv
def cudf_pandas_df():
    # Defina o caminho para o seu arquivo CSV
    file_path = "data/measurements_pandas.txt"
    
    # Tamanho do chunk (você pode ajustar conforme necessário)
    chunk_size = 100_000_000  # por exemplo, 1 milhão de linhas por chunk
    
    # Inicialize um DataFrame vazio para armazenar os resultados
    results_df = pd.DataFrame()
    
    # Loop através do arquivo em chunks e calcule as estatísticas
    for chunk in pd.read_csv(file_path,sep=';', header=None, names=['city', 'temp'], chunksize=chunk_size):
        # Calcular as estatísticas por grupo
        grouped_stats = chunk.groupby('city').agg({'temp': ['max','min','mean']}).reset_index()
        # Concatenar os resultados ao DataFrame principal
        results_df = pd.concat([results_df, grouped_stats], ignore_index=True)
    # Removendo o level city    
    results_df = results_df.droplevel(0,axis=1)
    # Renomeando o level 0 para city
    results_df.rename(columns={'':'city'}, inplace=True)
    # Fazendo o groupby geral.
    results_df.groupby('city').agg({ 'max':'max', 'min':'min', 'mean':'mean'}).round(2).sort_values('city')    
    # Resultados finais
    return display(results_df)

if __name__== "__main__":
    print(cudf_pandas_df())
    

The cudf.pandas extension is already loaded. To reload it, use:
  %reload_ext cudf.pandas


,city,max,min,mean
0,A Coruña,52.7,-88.1,-13.647368
1,A Yun Pa,50.6,-85.3,-21.007407
2,Aabenraa,47.7,-88.5,-28.036842
3,Aachen,38.8,-77.3,-30.217391
4,Aadorf,50.2,-85.2,-13.600000
...,...,...,...,...
41338,’Tlat Bni Oukil,48.1,-81.7,-19.422222
41339,’s-Gravendeel,54.0,-75.7,-6.096429
41340,’s-Gravenzande,23.4,-82.3,-22.825000
41341,’s-Heerenberg,55.0,-88.7,-5.705556


Tempo total de execução: 0.818 segundos
None


### Documentação do Script cudf_pandas 

* **Para a leitura para 10 milhões de linha foi extremamente rápido, porém para 1 bilhão a memória não suportou.**

**Descrição:**

Este script demonstra como calcular a temperatura mínima, máxima e média para cada cidade em um conjunto de dados utilizando a biblioteca CuDF. O CuDF é uma biblioteca similar ao Pandas, porém otimizada para processamento em GPUs, possibilitando um desempenho significativamente superior.

**Importante:**

Este script **não é recomendado** para processar um bilhão de linhas, pois a memória da GPU pode ser insuficiente. Para conjuntos de dados massivos, utilize a versão do script CuDF otimizada para Dask.

**Funcionalidade:**

**1. Leitura de dados:**

* Lê o arquivo CSV `data/measurements_pandas.txt` usando `cudf.read_csv`.
* Define `header=None` pois o arquivo não possui cabeçalho e `sep=;` como separador.
* Cria as colunas "city" e "temp".

**2. Cálculo de estatísticas:**

* Agrupa o DataFrame por "city" com `df.groupby('city')`.
* Calcula o mínimo, máximo e média da temperatura para cada cidade usando `agg` com uma lista de funções de agregação.

**3. Ordenação:**

* Ordena o DataFrame pelo índice (cidade) usando `sort_index()`.

**4. Exibição de resultados:**

* Imprime o DataFrame final com as estatísticas na tela.

**Características:**

* **Processamento em GPU:** O CuDF utiliza a GPU para realizar cálculos, proporcionando um aumento significativo de performance em comparação ao Pandas em CPUs.
* **Interface familiar:** A API do CuDF é similar ao Pandas, facilitando a migração de scripts existentes.

**Observações:**

* Certifique-se de ter o CuDF instalado e drivers Nvidia atualizados.
* O arquivo `data/measurements_pandas.txt` deve existir no caminho especificado.
* A memória da GPU pode ser insuficiente para processar grandes conjuntos de dados.

**Execução:**

1. **Navegue até o diretório do script:** `cd src`
2. **Execute o script:** `python cudf_pandas_df.py`

In [8]:
!python src/cudf_pandas_df.py

                  temp                 
                   min   max       mean
city                                   
A Coruña         -88.1  52.7 -13.647368
A Yun Pa         -85.3  50.6 -21.007407
Aabenraa         -88.5  47.7 -28.036842
Aachen           -77.3  38.8 -30.217391
Aadorf           -85.2  50.2 -13.600000
...                ...   ...        ...
’Tlat Bni Oukil  -81.7  48.1 -19.422222
’s-Gravendeel    -75.7  54.0  -6.096429
’s-Gravenzande   -82.3  23.4 -22.825000
’s-Heerenberg    -88.7  55.0  -5.705556
’s-Hertogenbosch -73.9  54.2  -5.000000

[41343 rows x 3 columns]
Tempo total de execução: 0.690 segundos


### Script cudf_pandas_df:

In [6]:
%%writefile src/cudf_pandas_df.py
# Script usando cudf == pandas mas rápido pq usa a GPU, porém a mem não suport procesar 1bilhão.
import cudf
from utils.decorators import timer_to_csv
@timer_to_csv
def cudf_pandas(filename):
    
    # Carregar o arquivo CSV e criar os cabeçalhos 'city' e 'temp'
    df = cudf.read_csv(filename, header=None, sep=';', names=['city', 'temp'])
    
    # Agrupar por 'city' e calcular o 'min', 'max' e 'mean' da coluna 'temp'
    grouped_df = df.groupby('city').agg({'temp': ['min', 'max', 'mean']})
    
    # Ordenar o DataFrame pelo índice (city)
    result = grouped_df.sort_index()     
    
    # retorna o resultado
    return print(result)

if __name__ == "__main__":
    filename = "data/measurements_pandas.txt"
    cudf_pandas(filename)


Overwriting src/cudf_pandas_df.py


In [48]:
%%time
import dask_cudf as dc

# Carregar o arquivo csv em um dataframe dask_cudf
df = dc.read_csv('data/measurements.txt',sep=';', header=None, names=['city', 'temp'], dtype=['str', 'float32'], blocksize='1024 Mib')

# Agrupar pela coluna 'city' e calcular min, max e mean da coluna 'temp'
grouped_df = df.groupby('city').agg({'temp': ['min', 'max', 'mean']}).compute()

# Ordenar pelo 'city'
sorted_df = grouped_df.sort_index()

# Imprimir as 5 primeiras e últimas linhas
display(sorted_df)
#print(sorted_df.tail(5))


TypeError: expected string or bytes-like object, got 'NoneType'

## Dask_cudf com GPU

#### Para utilização do dasck_cudf será necessária a instalação via pip conforme abaixo.

In [ ]:
# Instalação da lib via pip, via poetry apresentou erros.
!pip install --no-cache-dir --extra-index-url=https://pypi.nvidia.com dask-cudf-cu12 -q

In [5]:
# Instalação da lib dask_cuda
!pip install dask-cuda -q

In [1]:
#Reinicia o kernel jupyter
get_ipython().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

### Documentação do Script Dask-CuDF com Blocksize 1024

**Descrição:**

Este script demonstra como calcular a temperatura mínima, máxima e média para cada cidade em um conjunto de dados de um bilhão de linhas utilizando Dask-CuDF, combinando o poder do processamento paralelo do Dask com a aceleração por GPU do CuDF.

**Funcionalidade:**

**1. Configuração do Cluster GPU:**

* Cria um cluster local com um único nó utilizando a GPU disponível (`LocalCUDACluster`).
* Inicia um cliente Dask conectado ao cluster para distribuir o processamento.

**2. Leitura dos Dados:**

* Lê o arquivo CSV `data/measurements_pandas.txt` usando `dc.read_csv`.
* Define as seguintes opções:
    - `sep=';'` para indicar o separador de colunas.
    - `header=None` pois o arquivo não possui cabeçalho.
    - `names=['city', 'temp']` para nomear as colunas.
    - `dtype={'city': 'str', 'temp': 'float32'}` para especificar os tipos de dados.
    - `blocksize='1024 MiB'` para dividir o arquivo em partições de 1024 MiB para processamento paralelo.

**3. Cálculo de Estatísticas:**

* Agrupa o DataFrame por "city" usando `df.groupby('city')`.
* Calcula o máximo, mínimo e média da temperatura para cada cidade usando `agg` com uma lista de funções de agregação.
* Computa os resultados finais com `compute()`.
* Ordena o DataFrame pelo índice (cidade) usando `sort_index()`.

**4. Exibição de Resultados:**

* Imprime o DataFrame final contendo as estatísticas na tela.

**Características:**

* **Processamento em GPU Distribuído:** Combina Dask para paralelismo com CuDF para aceleração por GPU.
* **Blocksize para Particionamento:** Divide o arquivo em partições para melhor processamento paralelo.
* **Interface Familiar:** API similar ao Pandas, facilitando a adaptação.

**Observações:**

* Certifique-se de ter Dask, Dask-CUDA e CuDF instalados.
* A GPU deve ter memória suficiente para processar as partições de 1024 MiB.
* O arquivo `data/measurements_pandas.txt` deve existir no caminho especificado.

**Execução:**

1. **Navegue até o diretório do script:** `cd src`
2. **Execute o script:** `python dask_cudf_1024.py`

In [9]:
!python src/dask_cudf_1024.py

/home/jcnok/bootcamps/mytests/bootcamp-jornada-de-dados_2024/.venv/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34397 instead
  warnings.warn(
/home/jcnok/bootcamps/mytests/bootcamp-jornada-de-dados_2024/.venv/lib/python3.10/site-packages/dask_cuda/utils.py:170: UserWarning: Cannot get CPU affinity for device with index 0, setting default affinity
  warnings.warn(
                       temp                      
                        max        min       mean
city                                             
A Coruña          52.700001 -88.099998 -13.647368
A Yun Pa          50.599998 -85.300003 -21.007410
Aabenraa          47.700001 -88.500000 -28.036840
Aachen            38.799999 -77.300003 -30.217391
Aadorf            50.200001 -85.199997 -13.600000
...                     ...        ...        ...
’Tlat Bni Oukil   48.099998 -81.699997 -19.422223
’s-Gra

### Script Dasck_cudf_1024:

In [8]:
%%writefile src/dask_cudf_1024.py
# dask_cudf com blocksize 1024 => script para calcular min, max e mean em um bilhão de linhas usando a gpu rtx 3060ti.
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from utils.decorators import timer_to_csv
import dask_cudf as dc

@timer_to_csv
def dask_cudf_1024(filename):
    # Definindo o cluster para usar gpu
    with LocalCUDACluster() as cluster, Client(cluster) as client:
        # Carregar o arquivo csv em um dataframe dask_cudf
        df = dc.read_csv(filename,
                         sep=';', header=None,
                         names=['city', 'temp'], 
                         dtype={'city': 'str', 'temp': 'float32'},
                         blocksize='1024 MiB')
        
        # Agrupar pela coluna 'city' e calcular min, max e mean da coluna 'temp'
        result = df.groupby('city').agg({'temp': ['max','min','mean']}).compute().sort_index()
        print(result)
if __name__ == "__main__":
    filename= 'data/measurements_pandas.txt'
    dask_cudf_1024(filename)

Writing src/dask_cudf_1024.py


### Script Dask_cudf_256:

In [3]:
%%writefile src/dask_cudf_256.py
# dask_cudf com blocksize 256mb => script para caluclar min, max e mean em um bilhão de linhas usando a gpu rtx 3060ti.
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from utils.decorators import timer_to_csv
import dask_cudf as dc

@timer_to_csv
def dask_cudf_256(filename):
    # Definindo o cluster para usar gpu
    with LocalCUDACluster() as cluster, Client(cluster) as client:
        # Carregar o arquivo csv em um dataframe dask_cudf
        df = dc.read_csv(filename,
                         sep=';', header=None,
                         names=['city', 'temp'], 
                         dtype={'city': 'str', 'temp': 'float32'},
                         blocksize='256 MiB')
        
        # Agrupar pela coluna 'city' e calcular min, max e mean da coluna 'temp'
        result = df.groupby('city').agg({'temp': ['max','min','mean']}).compute().sort_index()
        print(result)
if __name__ == "__main__":
    filename= 'data/measurements_pandas.txt'
    dask_cudf_256(filename)

Overwriting src/dask_cudf_256.py


In [4]:
!python src/dask_cudf_256.py

/home/jcnok/bootcamps/mytests/bootcamp-jornada-de-dados_2024/.venv/lib/python3.10/site-packages/dask_cuda/utils.py:170: UserWarning: Cannot get CPU affinity for device with index 0, setting default affinity
  warnings.warn(
                       temp                      
                        max        min       mean
city                                             
A Coruña          52.700001 -88.099998 -13.647368
A Yun Pa          50.599998 -85.300003 -21.007410
Aabenraa          47.700001 -88.500000 -28.036840
Aachen            38.799999 -77.300003 -30.217391
Aadorf            50.200001 -85.199997 -13.600000
...                     ...        ...        ...
’Tlat Bni Oukil   48.099998 -81.699997 -19.422223
’s-Gravendeel     54.000000 -75.699997  -6.096428
’s-Gravenzande    23.400000 -82.300003 -22.825000
’s-Heerenberg     55.000000 -88.699997  -5.705555
’s-Hertogenbosch  54.200001 -73.900002  -4.999999

[41343 rows x 3 columns]
Tempo total de execução: 5.131 segundos


* **Alterando para 256 MiB a performance melhorou, mas ainda assim bem longe de superar o DuckDB.**
* **Conforme o benchmarch da documentação ele tem capacidade de superar o sgdb. Talvez alguma configuração pois ainda não li toda a documentação.**

## Gerando um Gráfico dos Resultados:

#### Segue abaixo o resultado do ranking geral de todos os testes realizados para calcular o min, max e média ordenado pela cidade em um bilhão de linhas.

### Resultado realizado para calcular o min, max e média ordenado pela cidade em um Milhão de linhas.

![png](./img/ranking.png)

* **obs:** Vale ressaltar que esse resultado só serve de parâmetro para minha máquina, com base nos scripts gerados aqui, certamente cabem espaços para otimização e em outros cenários os resultados podem ser bem divergentes.
* segue as configurações da minha máquina abaixo:

#### Memória:
Memory block size:       128M
Total online memory:      12G
Total offline memory:      0B

#### CPU:
Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  24
  On-line CPU(s) list:   0-23
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU E5-2670 v3 @ 2.30GHz
    CPU family:          6
    Model:               63
    Thread(s) per core:  2
    Core(s) per socket:  12
    Socket(s):           1

#### GPU:
RTX 3060ti 8Gbs

In [ ]:
# Instalação da lib para plotar o grafico.
!poetry add matplotlib

### Documentação do Script: Plotando Ranking de Tempo de Execução

**Descrição:**

Este script Python gera um gráfico de barras que mostra o ranking das funções em relação ao tempo de execução, ordenando-as do menor para o maior tempo.

**Funcionalidade:**

**1. Leitura dos Dados:**

* Lê o arquivo CSV `data/tempos_execucao.csv` que contém o nome da função e o tempo de execução (em segundos).
* Cada linha do arquivo CSV deve conter:
    - Nome da função (string).
    - Tempo de execução no formato `X segundos` ou `Y minutos e Z segundos`.

**2. Extração e Conversão:**

* Extrai o nome da função e o tempo de execução de cada linha.
* Converte o tempo de execução para segundos, considerando minutos e segundos.
* Armazena os dados em uma lista de tuplas (`(nome_funcao, tempo_total)`).

**3. Ordenação:**

* Ordena a lista de tuplas pelo tempo de execução (tempo_total) em ordem crescente.

**4. Plotagem do Gráfico:**

* Cria um gráfico de barras com a biblioteca `matplotlib`.
* As funções (ordenadas) são dispostas no eixo Y e o tempo de execução (em segundos) no eixo X.
* As barras são coloridas em azul celeste.
* O título do gráfico indica "Tempo de Execução das Funções (Ordenado)".
* O eixo X é rotulado como "Tempo de Execução (segundos)" e o eixo Y como "Função".
* Uma grade é adicionada ao eixo X para facilitar a leitura.
* O tempo de execução de cada função é exibido acima da barra respectiva.

**5. Salva o Gráfico:**

* Salva o gráfico para visualização dos resultados na pasta img/ranking.png.

**Observações:**

* O arquivo `data/tempos_execucao.csv` deve existir no caminho especificado.
 
**Exemplo de Uso:**

1. **Navegue até o diretório do script:** `cd src`
2. **Execute o script:** `python plot_ranking.py`


In [20]:
!python src/plot_ranking.py

Gráfico salvo com sucesso na pasta:img/ranking.png


### Script para plotar os resultados:

In [19]:
%%writefile src/plot_ranking.py
import csv
import matplotlib.pyplot as plt

def plot_ranking_tempo_execucao(arquivo_csv):
    # Leitura dos dados do arquivo CSV
    dados = []

    with open(arquivo_csv, 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            nome_funcao = row[0]
            tempo = row[1].split()  # Separa minutos e segundos
            if len(tempo) == 2 and tempo[1] == 'segundos':  # Se houver apenas segundos
                tempo_total = float(tempo[0])
            elif len(tempo) == 4:  # Se houver minutos e segundos
                minutos = float(tempo[0])
                segundos = float(tempo[2])
                tempo_total = minutos * 60 + segundos  # Converte minutos para segundos
            else:
                raise ValueError(f'Formato de tempo inválido: {row[1]}')
            dados.append((nome_funcao, tempo_total))

    # Ordena os dados pelo tempo de execução
    dados_ordenados = sorted(dados, key=lambda x: x[1])

    # Extrai os nomes das funções e os tempos de execução ordenados
    nomes_funcoes_ordenados = [item[0] for item in dados_ordenados]
    tempos_execucao_ordenados = [item[1] for item in dados_ordenados]

    # Plotagem do gráfico
    plt.figure(figsize=(10, 6))
    plt.barh(nomes_funcoes_ordenados, tempos_execucao_ordenados, color='skyblue')
    plt.xlabel('Tempo de Execução (segundos)')
    plt.ylabel('Função')
    plt.title('Tempo de Execução das Funções (Ordenado)')
    plt.grid(axis='x')
    plt.tight_layout()

    # Adicionando os valores nas barras
    for i, valor in enumerate(tempos_execucao_ordenados):
        plt.text(valor, i, f'{valor:.2f} s', va='center')

    # Exibindo o gráfico
    # plt.show()
    plt.savefig('./img/ranking.png')
    print('Gráfico salvo com sucesso na pasta:img/ranking.png')

# Exemplo de uso da função
if __name__ == "__main__":
    arquivo_csv = 'data/tempos_execucao.csv'
    plot_ranking_tempo_execucao(arquivo_csv)


Overwriting src/plot_ranking.py
